In [ ]:
# imports
import torch
import numpy as np
import pyvista as pv
from tools import load_experiment_models
from tools import uncertainty_plot

In [ ]:
# Example usage:
device='cuda'
config_path = 'configs/experiment2.conf'  # Path to the config file

# Load the models for the specified experiment
models = load_experiment_models(config_path, device=device)

In [ ]:
# extracting the densities
N = 100
scale = 1.5

x = torch.linspace(-scale, scale, N)
y = torch.linspace(-scale, scale, N)
z = torch.linspace(-scale, scale, N)

x, y, z = torch.meshgrid((x, y, z))

xyz = torch.cat((x.reshape(-1, 1),
                 y.reshape(-1, 1),
                 z.reshape(-1, 1)), dim=1)

In [ ]:
# arithmetic mean of each point (x,y,z) in our 3D grid
densities = []
for model in models:

    with torch.no_grad():
        _, density = model.forward(xyz.to(device), torch.zeros_like(xyz).to(device))
    
    #density = density.cpu().numpy().reshape(N, N, N)
    density = density.view(N, N, N)
    densities.append(density)

densities_tensor = torch.stack(densities)
mean_density = torch.mean(densities_tensor, dim=0) # calculates the arithmetic mean for every point

In [ ]:
# for all densities across the 7 models, substract the model density from the mean
squared_diffs = []

for model_density in densities: # for each model, subtact the mean density from each element.

    squared_diffs.append(torch.square(torch.sub(model_density, mean_density))) # squared diffference for all points

tensor_sum = torch.sum(torch.stack(squared_diffs), dim=0)
divisor = len(models) - 1
sample_variance = torch.div(tensor_sum, divisor)
coeff_variation = torch.div(torch.sqrt(sample_variance), mean_density)

In [ ]:
coeff_variation = coeff_variation.cpu().numpy().reshape(N, N, N)
sample_variance = sample_variance.cpu().numpy().reshape(N, N, N)
sf_mean_density = mean_density.cpu().numpy().reshape(N, N, N)

In [ ]:
pts, values = uncertainty_plot(scalar_field = sf_mean_density, scalars=None, pts=None, threshold=10)

In [ ]:
from scipy.spatial import KDTree

# Note: neighbor idx returns the index in the array of all pts of the neighbors.
map_tree = KDTree(pts)
neighbor_idx = map_tree.query_ball_point(x=pts, r=4) # returns list of indices of neighbors of x

In [ ]:
CoV = [] # list of coefficent of variation for each point based on variation of neighbor densities
for i in range(len(neighbor_idx)):
    neighbors = neighbor_idx[i] # [len(neighbors of that point)]

    surrounding_densities = [] # reinit list of neighbor for each point in the nerf
    for j in range(len(neighbors)):
        surrounding_densities.append(values[neighbors[j]]) # [n] densitity at each index in neighbors

    densities_tensor = torch.Tensor(surrounding_densities)
    mean_density = torch.mean(densities_tensor, dim=0) # calculates the arithmetic mean for every point

    squared_diffs = []
    for d in densities_tensor: # for each point, subtact the mean density from each element.

        squared_diffs.append(torch.square(torch.sub(d, mean_density))) # squared diffference for all points
    
    tensor_sum = torch.sum(torch.stack(squared_diffs), dim=0)
    divisor = len(models) - 1
    sample_variance = torch.div(tensor_sum, divisor)
    coeff_variation = torch.div(torch.sqrt(sample_variance), mean_density)
    CoV.append(coeff_variation)
    

In [ ]:
uncertainty_plot(scalar_field=None, scalars=np.array(CoV), pts=pts, threshold=3.5)

In [ ]:
object_mask = np.load('monkey_mask.npy') # [3,len(points)]
x_coords, y_coords, z_coords = object_mask
mask_pts = np.stack((x_coords, y_coords, z_coords), axis=-1)

In [ ]:
uncertainty_plot(scalar_field=None, scalars=np.array(CoV), pts=mask_pts, threshold=3.5)